In [ ]:
'''
since the dataset is from kaggle we can download it with opendataset but we need an access token which is expire after a while
!pip install opendatasets
import opendatasets as od
od.download("https://www.kaggle.com/datasets/pigment/big-sales-data")
'''
!wget https://users.itk.ppke.hu/~pasda2/Books_rating.csv.zip
!unzip Books_rating.csv.zip

!pip install pyspark
!pip install sparkmeasure



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=60256faff603135feb832417d281b317e4a92361e617881299805c006b412a80
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
#Linear Regression

import csv
import time
# We can set here how many times we want to run the algorithms
for i in range(5):
    %reset -f
    import csv
    import time
    from pyspark.sql import SparkSession
    from pyspark.context import SparkContext
    from pyspark import SparkConf
    conf = SparkConf().setAppName("Regression test")#\
    #.set('spark.sql.files.maxPartitionBytes','62914560')
    #.set('spark.sql.files.maxPartitionBytes','402653184')

    spark = SparkSession \
        .builder \
        .config(conf=conf)\
        .config("spark.ui.port", "4050") \
        .getOrCreate()
    fulltime=time.time()
    logs=[]
    log={}
    st=time.time()
    df=spark.read.csv("Books_rating.csv", header=True, inferSchema=True)
    log["algorithm"]="linear_regression"
    log["read_csv"]=time.time()-st
    print(time.time()-st)
    print(df.rdd.getNumPartitions())
    log["number_of_partitions"]=df.rdd.getNumPartitions()
    st=time.time()
    from pyspark.sql.types import *
    columns = []

    for i in df.dtypes:
        columns.append(i[0])
    print(columns)

    # Write a custom function to convert the data type of DataFrame columns
    def convertColumn(df, names, newType):
        for name in names:
            df = df.withColumn(name, df[name].cast(newType))
        return df


    # Conver the df columns to FloatType()
    df = convertColumn(df, columns, FloatType())

    df.fillna(value=0)
    df.printSchema()
    print(time.time()-st)

    from traitlets.traitlets import Float
    df=df.fillna(value=0)
    df.show()
    import numpy as np
    from pyspark.sql import functions as F
    def convertColumnNeg(df, names):
        for name in names:
            df = df.withColumn(name,F.when(df[name]<0,0).otherwise(F.col(name)))
        return df

    df=convertColumnNeg(df,columns)
    df=df.replace([np.inf, -np.inf], 0)
    print(time.time()-st)

    from pyspark.ml.regression import DecisionTreeRegressor
    from pyspark.ml.feature import VectorAssembler
    from pyspark.ml.evaluation import RegressionEvaluator
    incols=columns
    print(incols)
    incols.remove("review/score")
    assembler = VectorAssembler(inputCols=incols,outputCol="features")
    df = assembler.transform(df)
    df.printSchema()
    final_data = df.select("features", F.col("review/score").alias("score"))
    final_data.printSchema()
    log["data_convertion_time"]=time.time()-st
    st=time.time()
    # Import StandardScaler
    from pyspark.ml.feature import StandardScaler

    # Initialize the standardScaler
    standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")

    # Fit the DataFrame to the scaler
    scaler = standardScaler.fit(final_data)
    # Transform the data in `df` with the scaler
    scaled_df = scaler.transform(final_data)
    log["data_scale_time"]=time.time()-st
    st=time.time()
    print(time.time()-st)
    train_data, test_data = scaled_df.randomSplit([.8,.2],seed=1234)
    log["data_split_time"]=time.time()-st
    st=time.time()
    print(time.time()-st)
    from pyspark.ml.regression import LinearRegression
    lr = LinearRegression(featuresCol = 'features', labelCol='score', maxIter=10, regParam=0.3, elasticNetParam=0.8)
    lr_model = lr.fit(train_data)
    log["model_fit_time"]=time.time()-st
    st=time.time()
    print(time.time()-st)
    lr_predictions = lr_model.transform(test_data)
    log["prediction_time"]=time.time()-st
    st=time.time()
    from pyspark.ml.evaluation import RegressionEvaluator
    lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                     labelCol="score",metricName="r2")
    print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))
    test_result = lr_model.evaluate(test_data)
    print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)
    log["evaulation_time"]=time.time()-st
    log["full_time"]=time.time()-fulltime
    logs.append(log)
    print(time.time()-st)
    trainingSummary = lr_model.summary
    print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
    print("r2: %f" % trainingSummary.r2)
    print("numIterations: %d" % trainingSummary.totalIterations)
    print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
    #trainingSummary.residuals.show()
    print(logs)

    with open("logs.csv", mode='a', newline='') as file:
        # Create a CSV writer object
        csv_writer = csv.writer(file)

        # If the file is empty, write the header
        if file.tell() == 0:
            csv_writer.writerow(list(log.keys()))
        # Write the new data to the end of the CSV file
        csv_writer.writerow(list(log.values()))


    del df
    del train_data
    del test_data
    del scaled_df
    del final_data
    del columns
    del incols
    del log

    #Decision Tree Regression

    fulltime=time.time()
    log={}
    st=time.time()
    df=spark.read.csv("Books_rating.csv", header=True, inferSchema=True)
    log["algorithm"]="decision_tree_regression"
    log["read_csv"]=time.time()-st
    print(time.time()-st)
    print(df.rdd.getNumPartitions())
    log["number_of_partitions"]=df.rdd.getNumPartitions()
    st=time.time()
    from pyspark.sql.types import *
    columns = []

    for i in df.dtypes:
        columns.append(i[0])
    print(columns)

    # Write a custom function to convert the data type of DataFrame columns
    def convertColumn(df, names, newType):
        for name in names:
            df = df.withColumn(name, df[name].cast(newType))
        return df


    # Conver the `df` columns to `FloatType()`
    df = convertColumn(df, columns, FloatType())

    df.fillna(value=0)
    df.printSchema()
    print(time.time()-st)

    from traitlets.traitlets import Float
    df=df.fillna(value=0)
    df.show()
    import numpy as np
    from pyspark.sql import functions as F
    def convertColumnNeg(df, names):
        for name in names:
            df = df.withColumn(name,F.when(df[name]<0,0).otherwise(F.col(name)))
        return df

    df=convertColumnNeg(df,columns)
    df=df.replace([np.inf, -np.inf], 0)
    print(time.time()-st)

    from pyspark.ml.regression import DecisionTreeRegressor
    from pyspark.ml.feature import VectorAssembler
    from pyspark.ml.evaluation import RegressionEvaluator
    incols=columns
    print(incols)
    incols.remove("review/score")
    assembler = VectorAssembler(inputCols=incols,outputCol="features")
    df = assembler.transform(df)
    df.printSchema()
    final_data = df.select("features", F.col("review/score").alias("score"))
    final_data.printSchema()
    log["data_convertion_time"]=time.time()-st
    st=time.time()
    # Import `StandardScaler`
    from pyspark.ml.feature import StandardScaler

    # Initialize the `standardScaler`
    standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")

    # Fit the DataFrame to the scaler
    scaler = standardScaler.fit(final_data)
    # Transform the data in `df` with the scaler
    scaled_df = scaler.transform(final_data)
    log["data_scale_time"]=time.time()-st
    st=time.time()
    print(time.time()-st)
    train_data, test_data = scaled_df.randomSplit([.8,.2],seed=1234)
    log["data_split_time"]=time.time()-st
    st=time.time()
    print(time.time()-st)
    from pyspark.ml.regression import LinearRegression
    dr=DecisionTreeRegressor(featuresCol = 'features', labelCol='score')
    model=dr.fit(train_data)
    log["model_fit_time"]=time.time()-st
    st=time.time()
    print(time.time()-st)
    dr_predictions =model.transform(test_data)
    log["prediction_time"]=time.time()-st
    st=time.time()
    print(time.time()-st)

    from pyspark.ml.evaluation import RegressionEvaluator
    dr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                     labelCol="score")
    print("R Squared (R2) on test data = %g" % dr_evaluator.evaluate(dr_predictions,{dr_evaluator.metricName: "r2"}))
    print("Root Mean Squared Error (RMSE) on test data = %g" % dr_evaluator.evaluate(dr_predictions,{dr_evaluator.metricName: "rmse"}))
    print(time.time()-st)
    log["evaulation_time"]=time.time()-st
    log["full_time"]=time.time()-fulltime
    logs.append(log)
    print(logs)

    with open("logs.csv", mode='a', newline='') as file:
        # Create a CSV writer object
        csv_writer = csv.writer(file)

        # If the file is empty, write the header
        if file.tell() == 0:
            csv_writer.writerow(list(log.keys()))
        # Write the new data to the end of the CSV file
        csv_writer.writerow(list(log.values()))


    del df
    del train_data
    del test_data
    del scaled_df
    del final_data
    del columns
    del incols
    del log
    del model






    #Random Forest Regression

    fulltime=time.time()
    log={}
    st=time.time()
    df=spark.read.csv("Books_rating.csv", header=True, inferSchema=True)
    log["algorithm"]="random_forest_regression"
    log["read_csv"]=time.time()-st
    log["number_of_partitions"]=df.rdd.getNumPartitions()
    st=time.time()

    from pyspark.sql.types import *
    columns = []

    for i in df.dtypes:
        columns.append(i[0])
    print(columns)

    # Write a custom function to convert the data type of DataFrame columns
    def convertColumn(df, names, newType):
        for name in names:
            df = df.withColumn(name, df[name].cast(newType))
        return df


    # Conver the `df` columns to `FloatType()`
    df = convertColumn(df, columns, FloatType())

    df.fillna(value=0)
    df.printSchema()
    print(time.time()-st)

    from traitlets.traitlets import Float
    df=df.fillna(value=0)
    df.show()
    import numpy as np
    from pyspark.sql import functions as F
    def convertColumnNeg(df, names):
        for name in names:
            df = df.withColumn(name,F.when(df[name]<0,0).otherwise(F.col(name)))
        return df

    df=convertColumnNeg(df,columns)
    df=df.replace([np.inf, -np.inf], 0)
    print(time.time()-st)

    from pyspark.ml.regression import RandomForestRegressor
    from pyspark.ml.feature import VectorAssembler
    from pyspark.ml.evaluation import RegressionEvaluator
    incols=columns
    print(incols)
    incols.remove("review/score")
    assembler = VectorAssembler(inputCols=incols,outputCol="features")
    df = assembler.transform(df)
    df.printSchema()
    final_data = df.select("features", F.col("review/score").alias("score"))
    final_data.printSchema()
    log["data_convertion_time"]=time.time()-st
    st=time.time()
    # Import `StandardScaler`
    from pyspark.ml.feature import StandardScaler

    # Initialize the `standardScaler`
    standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")

    # Fit the DataFrame to the scaler
    scaler = standardScaler.fit(final_data)
    # Transform the data in `df` with the scaler
    scaled_df = scaler.transform(final_data)
    log["data_scale_time"]=time.time()-st
    st=time.time()
    print(time.time()-st)
    train_data, test_data = scaled_df.randomSplit([.8,.2],seed=1234)
    log["data_split_time"]=time.time()-st
    st=time.time()

    rf = RandomForestRegressor(featuresCol="features",labelCol="score")
    model=rf.fit(train_data)
    log["model_fit_time"]=time.time()-st
    st=time.time()
    print(time.time()-st)
    predictions=model.transform(test_data)
    log["prediction_time"]=time.time()-st
    st=time.time()
    # Select (prediction, true label) and compute test error
    evaluator = RegressionEvaluator(labelCol="score", predictionCol="prediction")
    print("R Squared (R2) on test data = %g" % evaluator.evaluate(predictions,{evaluator.metricName: "r2"}))
    print("Root Mean Squared Error (RMSE) on test data = %g" % evaluator.evaluate(predictions,{evaluator.metricName: "rmse"}))
    log["evaulation_time"]=time.time()-st
    log["full_time"]=time.time()-fulltime
    logs.append(log)
    print(logs)
    print(time.time()-st)

    with open("logs.csv", mode='a', newline='') as file:
        # Create a CSV writer object
        csv_writer = csv.writer(file)

        # If the file is empty, write the header
        if file.tell() == 0:
            csv_writer.writerow(list(log.keys()))
        # Write the new data to the end of the CSV file
        csv_writer.writerow(list(log.values()))



    del df
    del train_data
    del test_data
    del scaled_df
    del final_data
    del columns
    del incols
    del log
    del model






    #Gradient Boosted Regression
    fulltime=time.time()
    log={}
    st=time.time()
    df=spark.read.csv("Books_rating.csv", header=True, inferSchema=True)
    log["algorithm"]="gradient_boosted_regression"
    log["read_csv"]=time.time()-st
    log["number_of_partitions"]=df.rdd.getNumPartitions()
    st=time.time()

    from pyspark.sql.types import *
    columns = []

    for i in df.dtypes:
        columns.append(i[0])
    print(columns)

    # Write a custom function to convert the data type of DataFrame columns
    def convertColumn(df, names, newType):
        for name in names:
            df = df.withColumn(name, df[name].cast(newType))
        return df


    # Conver the `df` columns to `FloatType()`
    df = convertColumn(df, columns, FloatType())

    df.fillna(value=0)
    df.printSchema()
    print(time.time()-st)

    from traitlets.traitlets import Float
    df=df.fillna(value=0)
    df.show()
    import numpy as np
    from pyspark.sql import functions as F
    def convertColumnNeg(df, names):
        for name in names:
            df = df.withColumn(name,F.when(df[name]<0,0).otherwise(F.col(name)))
        return df

    df=convertColumnNeg(df,columns)
    df=df.replace([np.inf, -np.inf], 0)
    print(time.time()-st)

    from pyspark.ml.regression import GBTRegressor
    from pyspark.ml.feature import VectorAssembler
    from pyspark.ml.evaluation import RegressionEvaluator
    incols=columns
    print(incols)
    incols.remove("review/score")
    assembler = VectorAssembler(inputCols=incols,outputCol="features")
    df = assembler.transform(df)
    df.printSchema()
    final_data = df.select("features", F.col("review/score").alias("score"))
    final_data.printSchema()
    log["data_convertion_time"]=time.time()-st
    st=time.time()

    # Import `StandardScaler`
    from pyspark.ml.feature import StandardScaler

    # Initialize the `standardScaler`
    standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")

    # Fit the DataFrame to the scaler
    scaler = standardScaler.fit(final_data)
    # Transform the data in `df` with the scaler
    scaled_df = scaler.transform(final_data)
    log["data_scale_time"]=time.time()-st
    st=time.time()
    print(time.time()-st)
    train_data, test_data = scaled_df.randomSplit([.8,.2],seed=1234)
    print(time.time()-st)
    log["data_split_time"]=time.time()-st
    st=time.time()

    gbtr = GBTRegressor(featuresCol="features",labelCol="score")
    model=gbtr.fit(train_data)
    log["model_fit_time"]=time.time()-st
    st=time.time()
    print(time.time()-st)
    predictions=model.transform(test_data)
    log["prediction_time"]=time.time()-st
    st=time.time()
    # Select (prediction, true label) and compute test error
    evaluator = RegressionEvaluator(labelCol="score", predictionCol="prediction")
    print("R Squared (R2) on test data = %g" % evaluator.evaluate(predictions,{evaluator.metricName: "r2"}))
    print("Root Mean Squared Error (RMSE) on test data = %g" % evaluator.evaluate(predictions,{evaluator.metricName: "rmse"}))
    print(time.time()-st)
    log["evaulation_time"]=time.time()-st
    log["full_time"]=time.time()-fulltime
    logs.append(log)
    print(logs)
    print(time.time()-st)
    with open("logs.csv", mode='a', newline='') as file:
        # Create a CSV writer object
        csv_writer = csv.writer(file)

        # If the file is empty, write the header
        if file.tell() == 0:
            csv_writer.writerow(list(log.keys()))
        # Write the new data to the end of the CSV file
        csv_writer.writerow(list(log.values()))


    del df
    del train_data
    del test_data
    del scaled_df
    del final_data
    del columns
    del incols
    del log
    del model





    #NaiveBayes
    fulltime=time.time()
    log={}
    st=time.time()
    df=spark.read.csv("Books_rating.csv", header=True, inferSchema=True)
    log["algorithm"]="naive_bayes"
    log["read_csv"]=time.time()-st
    log["number_of_partitions"]=df.rdd.getNumPartitions()
    st=time.time()

    from pyspark.sql.types import *
    columns = []

    for i in df.dtypes:
        columns.append(i[0])
    print(columns)

    # Write a custom function to convert the data type of DataFrame columns
    def convertColumn(df, names, newType):
        for name in names:
            df = df.withColumn(name, df[name].cast(newType))
        return df


    # Conver the `df` columns to `FloatType()`
    df = convertColumn(df, columns, FloatType())

    df.fillna(value=0)
    df.printSchema()
    print(time.time()-st)

    from traitlets.traitlets import Float
    df=df.fillna(value=0)
    df.show()
    import numpy as np
    from pyspark.sql import functions as F
    def convertColumnNeg(df, names):
        for name in names:
            df = df.withColumn(name,F.when(df[name]<0,0).otherwise(F.col(name)))
        return df

    df=convertColumnNeg(df,columns)
    df=df.replace([np.inf, -np.inf], 0)
    print(time.time()-st)

    from pyspark.ml.feature import VectorAssembler
    from pyspark.ml.evaluation import RegressionEvaluator
    incols=columns
    print(incols)
    incols.remove("review/score")
    assembler = VectorAssembler(inputCols=incols,outputCol="features")
    df = assembler.transform(df)
    df.printSchema()
    final_data = df.select("features", F.col("review/score").alias("score"))
    final_data.printSchema()
    log["data_convertion_time"]=time.time()-st
    st=time.time()
    # Import `StandardScaler`
    from pyspark.ml.feature import StandardScaler

    # Initialize the `standardScaler`
    standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")

    # Fit the DataFrame to the scaler
    scaler = standardScaler.fit(final_data)
    # Transform the data in `df` with the scaler
    scaled_df = scaler.transform(final_data)
    log["data_scale_time"]=time.time()-st
    st=time.time()
    print(time.time()-st)
    train_data, test_data = scaled_df.randomSplit([.8,.2],seed=1234)
    log["data_split_time"]=time.time()-st
    st=time.time()
    print(time.time()-st)

    from pyspark.ml.classification import NaiveBayes
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    nb = NaiveBayes(smoothing=1.0, modelType="multinomial",labelCol="score")
    model = nb.fit(train_data)
    log["model_fit_time"]=time.time()-st
    st=time.time()
    print(time.time()-st)
    predictions=model.transform(test_data)
    log["prediction_time"]=time.time()-st
    st=time.time()
    # Select (prediction, true label) and compute test error
    evaluator = MulticlassClassificationEvaluator(labelCol="score", predictionCol="prediction",metricName="accuracy")
    accuracy = evaluator.evaluate(predictions)
    print("Test set accuracy = " + str(accuracy))
    print(time.time()-st)
    log["evaulation_time"]=time.time()-st
    log["full_time"]=time.time()-fulltime
    logs.append(log)
    print(logs)
    print(time.time()-st)
    with open("logs.csv", mode='a', newline='') as file:
        # Create a CSV writer object
        csv_writer = csv.writer(file)

        # If the file is empty, write the header
        if file.tell() == 0:
            csv_writer.writerow(list(log.keys()))
        # Write the new data to the end of the CSV file
        csv_writer.writerow(list(log.values()))

    del df
    del train_data
    del test_data
    del scaled_df
    del final_data
    del columns
    del incols
    del log
    del model
    del logs

54.401153802871704
22
['Id', 'Title', 'Price', 'User_id', 'profileName', 'review/helpfulness', 'review/score', 'review/time', 'review/summary', 'review/text']
root
 |-- Id: float (nullable = true)
 |-- Title: float (nullable = true)
 |-- Price: float (nullable = true)
 |-- User_id: float (nullable = true)
 |-- profileName: float (nullable = true)
 |-- review/helpfulness: float (nullable = true)
 |-- review/score: float (nullable = true)
 |-- review/time: float (nullable = true)
 |-- review/summary: float (nullable = true)
 |-- review/text: float (nullable = true)

1.1308960914611816
+-----------+-----+-----+-------+-----------+------------------+------------+-----------+--------------+-----------+
|         Id|Title|Price|User_id|profileName|review/helpfulness|review/score|review/time|review/summary|review/text|
+-----------+-----+-----+-------+-----------+------------------+------------+-----------+--------------+-----------+
|1.8829312E9|  0.0|  0.0|    0.0|        0.0|              

Py4JJavaError: An error occurred while calling o1923.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 252.0 failed 1 times, most recent failure: Lost task 2.0 in stage 252.0 (TID 4839) (1863e1f4e1c7 executor driver): java.lang.RuntimeException: Labels MUST be Integers, but got 19.950000762939453
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.project_doConsume_2$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenEvaluatorFactory$WholeStageCodegenPartitionEvaluator$$anon$1.hasNext(WholeStageCodegenEvaluatorFactory.scala:43)
	at org.apache.spark.sql.execution.aggregate.ObjectAggregationIterator.processInputs(ObjectAggregationIterator.scala:166)
	at org.apache.spark.sql.exe